In [367]:
import os
import sklearn
import pandas as pd
import csv

In [368]:
n_gram_value = 5

In [369]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [370]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [371]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [372]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [373]:
'''def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list   ''' 

'def preprocessing(possible_words,doc_id):    \n    suffix_list = getListFromFiles("company-suffixes.txt")\n    prefix_list = getListFromFiles("company-prefixes.txt")    \n    common_words = getListFromFiles("common-words.txt")\n    preprocessed_list =[]    \n    marked_list=[]\n    word_number=len(possible_words)\n    for i in range(5,0,-1):\n        word_count=0\n        ngram_list = get_ngrams(possible_words,i)\n        for ngram in ngram_list:            \n            #print ngram   \n            \n            #if markup is both at the beginning or at the end of the word group accept and label as 1\n            if(len(ngram)>1 and \'<markup>\' in ngram[0] and \'</markup>\' in ngram[len(ngram)-1] and\n              all((word[0].isupper() or "markup" in word) for word in ngram) and \n              ["<markup>" in word for word in ngram].count(True)==1):\n                preprocessed_list.append([\' \'.join(ngram),doc_id,word_count,word_count+i-1,1])\n            \n            #get all

In [374]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [375]:
def generateFeatures(filename, processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    final_processed_list = []
    splittedFilename = filename.split("-")
    i=0
    print "possible_words- "  
    print possible_words
    for company in processed_list:
        i+=1
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        #Feature 2: name occurs at the beginning of a company name
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        #Feature 3: name is a substring in the company name
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
         
        
        #Feature 4: name in filename
        isInFilename = 0
        splittedInstance = company[0].split()
        for sp in splittedInstance:
            if sp.lower() in splittedFilename:
                isInFilename = 1 
        
        
        #Feature 5-y - Check neibhouring words - check 3 words before and 3 words after
        starting_loc = company[2]
        ending_loc = company[3]
        words_before_starting_loc = []
        words_after_ending_loc = []
        j= starting_loc-1
        count = 1
        while j >= 0 and count <= 3:
            words_before_starting_loc.append(possible_words[company[2]-count])
            j-=1
            count+=1
        
        j = ending_loc+1
        count = 1
        while j < len(possible_words) and count <= 3:
            words_after_ending_loc.append(possible_words[company[3]+count])
            j+=1
            count+=1
        
        #Feature 5 - Check for word "based" before the company
        has_based = 0
        for word in words_before_starting_loc:
            if "based" in word.lower():
                has_based = 1
        
        #Feature 6 "'s" is after the company
        has_apostrophe_s = 0
        for word in words_after_ending_loc:
            if "'s" in word.lower():
                has_apostrophe_s = 1
        '''
        #Feature 7 "," is after/before the company
        has_comma = 0
        for word in words_after_ending_loc:
            if "," in word.lower():
                has_comma = 1
        
        for word in words_before_starting_loc:
            if "," in word.lower():
                has_comma = 1
        '''
        #Feature 8 designations before/after company
        desig = open('designations.txt').read().split('\n')
        desig_lower = [x.lower() for x in desig]
        has_desig = 0
        for word in words_after_ending_loc:
            if word.lower() in desig_lower:
                has_desig = 1
              
        for word in words_before_starting_loc:
            if word.lower() in desig_lower:
                has_desig = 1 
         
        #Feature 9 "said/told" in the before company
        has_said = 0
        for word in words_before_starting_loc:
            if "said" in word.lower() or "told" in word.lower():
                has_said = 1
        
        '''
        #Feature 10 Prune away Examples if previous/next word starts with a Capital
        surrounded_capital = 0
        if len(words_before_starting_loc) > 0:
            prev_word = words_before_starting_loc[0]
            if len(prev_word) > 0 and prev_word[0].isupper():
                surrounded_capital = 1
        if len( words_after_ending_loc) > 0:
            next_word = words_after_ending_loc[0]
            if len(next_word) > 0 and next_word[0].isupper():
                surrounded_capital = 1
        '''
          
                
        
        print "company:- " + company[0]
        print "words_after_ending_loc"
        print words_after_ending_loc
        print "words_before_starting_loc"
        print words_before_starting_loc
        
        company.extend([hasCompanyid, hasCompanyNameFirst, hasNameSubstring, isInFilename, has_based, has_apostrophe_s, has_desig, has_said])
        
            

In [376]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Testing_1")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    relevant_filename = file[4:-4]
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    #print processed_list_for_doc
    generateFeatures(relevant_filename, processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

possible_words- 
['American', 'Idol', 'the', 'most', 'popular', 'show', 'in', 'primetime', 'television', 'withstood', 'last', "week's", 'criticism', 'over', 'judging', 'and', 'returned', 'last', 'night', 'with', 'the', 'third-largest', 'audience', 'for', 'an', 'entertainment', 'show', 'this', 'season', 'With', 'about', '326', 'million', 'viewers', 'according', 'to', 'overnight', 'results', 'from', 'Nielsen', 'Media', 'Research', 'the', 'latest', 'Idol', 'trailed', 'only', 'the', 'two', 'Idol', 'shows', 'that', 'opened', 'its', 'sixth', 'season', 'on', 'Fox', 'last', 'week', 'with', 'audiences', 'of', '374', 'million', 'and', '37', 'million', 'and', 'the', 'National', 'Football', "League's", 'American', 'Conference', 'championship', 'game', 'on', 'CBS', 'last', 'Sunday', 'which', 'drew', '435', 'million', 'The', 'first', 'two', 'Idol', 'shows', 'were', 'the', 'second', 'and', 'third', 'most-watched', 'of', 'the', 'series', 'since', 'the', "show's", '2002', 'debut', 'They', 'also', 'spur

company:- Sontchi
words_after_ending_loc
['to', 'approve', 'the']
words_before_starting_loc
['ask', 'would', 'it']
company:- Spokespersons
words_after_ending_loc
['for', '<markup>Advanced', 'Marketing</markup>']
words_before_starting_loc
['15', 'Feb', 'held']
possible_words- 
['<markup>OAO', 'Aeroflot</markup>', 'AFLT', 'eastern', 'Europe', "'s", 'largest', 'airline', 'carried', '87', 'percent', 'more', 'passengers', 'last', 'year', 'as', 'the', "country's", 'growing', 'economy', 'boosted', 'travel', 'State-controlled', '<markup>Aeroflot</markup>', 'flew', '73', 'million', 'passengers', 'in', '2006', 'the', 'Moscow-based', 'airline', 'said', 'today', 'in', 'a', 'statement', 'The', 'company', 'handled', '02', 'percent', 'less', 'cargo', 'or', '145300', 'tons', 'last', 'year', 'The', 'load', 'factor', 'or', 'proportion', 'of', 'seats', 'filled', 'increased', 'to', '701', 'percent', 'from', '691', 'percent', 'in', '2005', '<markup>Aeroflot</markup>', 'is', 'attracting', 'more', 'travelers

['which', 'is', 'devoted']
words_before_starting_loc
['the', 'for', 'wish']
company:- JP Morgan Securities Inc
words_after_ending_loc
['wrote', 'today', 'in']
words_before_starting_loc
['at', 'analyst', 'an']
company:- Ameristar Casinos Inc
words_after_ending_loc
['ASCA', 'an', 'owner']
words_before_starting_loc
['of', 'Shares']
company:- Chairman Craig Neilsen
words_after_ending_loc
['led', 'to', 'speculation']
words_before_starting_loc
['and', 'shareholder', 'largest']
company:- Nasdaq Stock Market
words_after_ending_loc
['composite', 'trading', 'and']
words_before_starting_loc
['in', 'pm', '4']
company:- Colorado Iowa Missouri
words_after_ending_loc
['Mississippi', 'and', 'Nevada']
words_before_starting_loc
['in', 'casinos', 'operates']
company:- Iowa Missouri Mississippi
words_after_ending_loc
['and', 'Nevada', 'Neilsen']
words_before_starting_loc
['Colorado', 'in', 'casinos']
company:- Penn Capital Management
words_after_ending_loc
['that', 'owns', 'shares']
words_before_starting_

company:- Each
words_after_ending_loc
['warrant', 'will', 'be']
words_before_starting_loc
['28', 'March', 'on']
company:- Novelis
words_after_ending_loc
['Pragnya', 'Ram', 'company']
words_before_starting_loc
['of', 'buyout', 'leveraged']
company:- Ram
words_after_ending_loc
['company', 'spokeswoman', 'said']
words_before_starting_loc
['Pragnya', '<markup>Novelis</markup>', 'of']
company:- Hindalco
words_after_ending_loc
["'s", 'total', 'debt']
words_before_starting_loc
['15', 'Feb', 'said']
possible_words- 
['A', '<markup>Ceridian', 'Corp</markup>', 'unit', "didn't", 'infringe', 'a', 'Tennessee', "company's", 'patent', 'on', 'gift', 'cards', 'sold', 'at', 'store', 'checkout', 'stands', 'a', 'federal', 'jury', 'said', 'The', 'jury', 'in', 'Memphis', 'also', 'found', 'on', 'Nov', '3', 'that', '<markup>Barry', 'Fiala', 'Inc</markup>', "'s", 'patent', 'was', 'invalid', 'because', 'it', 'covered', 'an', 'obvious', 'variation', 'of', 'an', 'earlier', 'invention', 'and', 'unenforceable', 'be

company:- China's
words_after_ending_loc
['eastern', 'province', 'of']
words_before_starting_loc
['of', 'city', 'Maanshan']
company:- Magang
words_after_ending_loc
['the', "nation's", 'seventh-biggest']
words_before_starting_loc
['Anhui', 'of', 'province']
company:- Chinese
words_after_ending_loc
['demand', 'has', 'driven']
words_before_starting_loc
['as', 'imports', 'on']
company:- Chinese
words_after_ending_loc
['ore', 'typically', 'contains']
words_before_starting_loc
['records', 'to', 'prices']
company:- Magang
words_after_ending_loc
["'s", 'existing', 'iron']
words_before_starting_loc
['Australia', 'in', 'found']
company:- Shares
words_after_ending_loc
['of', '<markup>Magang</markup>', "'s"]
words_before_starting_loc
['said', 'statement', 'the']
company:- Magang
words_after_ending_loc
["'s", 'Hong', 'Kong-trade']
words_before_starting_loc
['of', 'Shares', 'said']
company:- Kong-trade
words_after_ending_loc
['unit', '<markup>Maanshan', 'Iron']
words_before_starting_loc
['Hong', "'s

...

company:- Reliant
words_after_ending_loc
['which', 'has', 'power']
words_before_starting_loc
['billion', '$234', 'to']
company:- US
words_after_ending_loc
['states', 'lost', 'sales']
words_before_starting_loc
['nine', 'plants', 'power']
company:- US
words_after_ending_loc
['power', 'market', 'tumbled']
words_before_starting_loc
['largest', 'the', 'LLC</markup>']
company:- Judge
words_after_ending_loc
['an', 'analyst', 'at']
words_before_starting_loc
['Nathan', 'said', 'demand']
company:- Staff
words_after_ending_loc
['sold', 'assets', 'and']
words_before_starting_loc
['Joel', 'Officer', 'Executive']
company:- Reliant
words_after_ending_loc
['last', 'week', 'said']
words_before_starting_loc
['2003', 'and', '2002']
company:- Staff
words_after_ending_loc
['will', 'retire', 'as']
words_before_starting_loc
['said', 'week', 'last']
company:- CEO
words_after_ending_loc
['in', 'May', 'and']
words_before_starting_loc
['as', 'retire', 'will']
company:- May
words_after_ending_loc
['and', 'be', 'r

['each', 'percent', '14']
company:- Tribune
words_after_ending_loc
['owner', 'of', '11']
words_before_starting_loc
['said', 'discounting', 'subscription']
company:- Times
words_after_ending_loc
['and', 'Chicago', '<markup>Tribune</markup>']
words_before_starting_loc
['Angeles', 'Los', 'the']
company:- Tribune
words_after_ending_loc
['Shares', 'of', '<markup>Tribune</markup>']
words_before_starting_loc
['Chicago', 'and', 'Times']
company:- Shares
words_after_ending_loc
['of', '<markup>Tribune</markup>', 'also']
words_before_starting_loc
['<markup>Tribune</markup>', 'Chicago', 'and']
company:- Tribune
words_after_ending_loc
['also', 'owner', 'of']
words_before_starting_loc
['of', 'Shares', '<markup>Tribune</markup>']
company:- Cubs
words_after_ending_loc
['baseball', 'team', 'and']
words_before_starting_loc
['Chicago', 'the', 'of']
company:- Exchange
words_after_ending_loc
['composite', 'trading', 'yesterday']
words_before_starting_loc
['Stock', 'York', 'New']
company:- Lazaroff
words_af

words_before_starting_loc
['ruble-based', 'The', '6']
company:- The Russian
words_after_ending_loc
['stock', 'market', 'the']
words_before_starting_loc
['155058', 'to', 'percent']
company:- New York
words_after_ending_loc
['Mercantile', 'Exchange', 'Futures']
words_before_starting_loc
['the', 'on', 'barrel']
company:- York Mercantile
words_after_ending_loc
['Exchange', 'Futures', 'gained']
words_before_starting_loc
['New', 'the', 'on']
company:- Mercantile Exchange
words_after_ending_loc
['Futures', 'gained', '24']
words_before_starting_loc
['York', 'New', 'the']
company:- Exchange Futures
words_after_ending_loc
['gained', '24', 'percent']
words_before_starting_loc
['Mercantile', 'York', 'New']
company:- Oil Price
words_after_ending_loc
['Expectation', 'There', 'is']
words_before_starting_loc
['$1170', 'to', 'percent']
company:- Price Expectation
words_after_ending_loc
['There', 'is', 'an']
words_before_starting_loc
['Oil', '$1170', 'to']
company:- Expectation There
words_after_ending_

company:- Unified Energy
words_after_ending_loc
['will', 'meet', 'with']
words_before_starting_loc
['of', 'CEO', 'Chubais']
company:- Russian
words_after_ending_loc
['stocks', 'rose', 'with']
words_before_starting_loc
[]
company:- Index
words_after_ending_loc
['gaining', 'the', 'most']
words_before_starting_loc
['RTS', 'the', 'with']
company:- Kommersant
words_after_ending_loc
['reported', 'President', 'Vladimir']
words_before_starting_loc
['System</markup>', 'Energy', 'Unified']
company:- Putin
words_after_ending_loc
['may', 'discuss', 'gas']
words_before_starting_loc
['Vladimir', 'President', 'reported']
company:- Russia's
words_after_ending_loc
['second-biggest', 'Internet', 'search']
words_before_starting_loc
['runs', 'which', 'Ltd</markup>']
company:- Internet
words_after_ending_loc
['search', 'engine', 'gained']
words_before_starting_loc
['second-biggest', "Russia's", 'runs']
company:- Potanin's
words_after_ending_loc
['company', 'bought', '49']
words_before_starting_loc
['Vladim

possible_words- 
['Shares', 'of', '<markup>Sacyr', 'Vallehermoso', 'SA</markup>', 'SYV', 'Spain', "'s", 'fifth-biggest', 'construction', 'company', 'had', 'their', 'sharpest-ever', 'gain', 'after', 'the', 'company', 'reported', 'a', '38', 'percent', 'jump', 'in', 'third-quarter', 'operating', 'profit', 'on', 'increased', 'orders', 'for', 'homes', 'and', 'roads', 'The', 'stock', 'closed', 'up', '713', 'euros', 'or', '15', 'percent', 'at', '54', 'euros', 'the', 'highest', 'since', 'the', 'company', 'was', 'formed', 'in', '2003', 'Shares', 'of', '<markup>Sacyr</markup>', 'have', 'almost', 'tripled', 'in', 'price', 'this', 'year', 'for', 'a', 'market', 'value', 'of', '154', 'billion', 'euros', '$20', 'billion', '<markup>Sacyr</markup>', 'gained', 'building', 'work', 'as', 'home', 'sales', 'rose', 'by', 'almost', 'a', 'third', 'in', 'the', 'nine', 'months', 'through', 'September', 'and', 'Spain', 'introduced', 'a', '249', 'billion-euro', 'plan', 'to', 'upgrade', 'roads', 'and', 'railways', 

In [377]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [378]:
["markup" in word for word in ngram].count(True)==2

True

In [379]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Latin America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")
country_names.append("Siberia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [380]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
cities.append('Montreal')
cities.append('Frankfurt')
cities.append('Zurich')
if "Compton" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [381]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'staff', 'secretary', 'attorney', 'jr', 'sr', 'officials', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief', 'commissioner', 'minister', 'magistrate', 'judge', 'chairman']


In [382]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


4528


In [383]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos: 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif inst[0].startswith("A ") or inst[0].startswith("An ") or inst[0].startswith("And ") or inst[0].startswith("This"):
             pruned_list.append(inst)
        elif inst[0].endswith("And") or inst[0].endswith("This"):
             pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print "HOLY MOLY!"
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", 'Cia Siderurgica Nacional SA', 'Cia Siderurgica Nacional', 'Vector Hospitality Plc', 'Vector', 'Victory']
2203
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY M

In [365]:
'''thefile = open('pruned_pos.txt', 'w')
for item in pruned_pos:
    #print item
    thefile.write("%s\n" % item)
len(pruned_pos)
print pruned_list'''

'thefile = open(\'pruned_pos.txt\', \'w\')\nfor item in pruned_pos:\n    #print item\n    thefile.write("%s\n" % item)\nlen(pruned_pos)\nprint pruned_list'

In [384]:
with open("test_pruned.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)